<center><h1>PyTorch Intro</h1></center>

<h2>Training a MNIST Classifier</h2>

In this notebook we will train a classifier that will classify images of the MNIST dataset. This dataset contains 28x28 pixel, 1-channel(grayscale) pictures that are labeled as one of the 10 digits.

Initially, we will import the libraries that we will use in order to load the data.

In [51]:
import torch
import torchvision     #Contains the necessary datasets
import torchvision.transforms as transforms #It is used to apply simple preprocessing steps into the dataset


<h2>Preprocessing</h2>


In [52]:
# Writing my own transform DivX that will divide a tensor by X
# I do not need it because everything is normalized
class DivX(object):
    def __init__(self, factor):
        self.factor = factor
    
    def __call__(self, sample):
        return torch.div(sample, self.factor)
    
# Defining transformation
transform = transforms.Compose([
    transforms.ToTensor()
    #DivX(255)
])

# Loading the dataset
trainset = torchvision.datasets.MNIST(root="./data", train=True, 
                                       download=True, transform=transform)

# Creating a DataLoader for that dataset.
# DataLoader is an iteratable that will help us train
# our classifier and get our batches automatically

trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                        shuffle=True, num_workers=2)

# Doing the same to create a test set
testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = (0,1,2,3,4,5,6,7,8,9)


Now we will print some images



<h2> Creating our network </h2>

Now we will create our convolutional neural network.

In order to create a neural network in PyTorch we should define a class that inherits <code>nn.Module</code> class and implements the contructor and the forward method.

In [53]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        # 1st arg = input channels, 2nd arg = output channels
        # 3rd arg = kernel size
        self.conv1 = nn.Conv2d(1, 4, (3,3))
        
        # There we define only kernel size
        self.pool1 = nn.MaxPool2d((2,2))
        self.conv2 = nn.Conv2d(4, 8, (3,3))
        self.pool2 = nn.MaxPool2d((2,2))
        
        # Now we will have the fully connected layers
        # Because in PyTorch there is not any flatten layer
        # we should compute the number of the input features 
        # of the flatten layer ourselves or do some computations.
        
        self.fc1 = nn.Linear(8 * 5 * 5, 50)
        self.fc3 = nn.Linear(50, 10)
    
    # x is the input batch
    # just passes x through networks operations
    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        
        x = x.view(-1, 8 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc3(x), dim=1)
        
        return x

net = Net()

#in order to use cuda
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)
net.to(device)

cuda:0


Net(
  (conv1): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(4, 8, kernel_size=(3, 3), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=200, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=10, bias=True)
)


After creating our network we should define our loss function and optimizer.


In [54]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0001)

<h2>Training the Network</h2>

In this section we will just iterate through the data loader and we will incrementally train our network

In [56]:
num_epochs = 10

for i in range(num_epochs):
    running_loss = 0
    
    for curr_batch in trainloader:
        vectors, labels = curr_batch[0].to(device), curr_batch[1].to(device)
        
        optimizer.zero_grad() # we will just make zero the gradients of optimizer
        
        outputs = net.forward(vectors) # we will do the forward pass for the current batch
        
        # Computing the loss
        loss    = criterion(outputs, labels)
        
        #Doing backward pass
        loss.backward()
        optimizer.step()
        
        running_loss = loss.item()
    
    print("Epoch", i, " Loss:", running_loss)
    
# In order to save our model
# PATH = './mnist_net.pth'
# torch.save(net.state_dict(), PATH)

Epoch 0  Loss: 1.4611575603485107
Epoch 1  Loss: 1.4611539840698242
Epoch 2  Loss: 1.4629478454589844
Epoch 3  Loss: 1.464242935180664
Epoch 4  Loss: 1.4611505270004272
Epoch 5  Loss: 1.4611502885818481
Epoch 6  Loss: 1.4611501693725586
Epoch 7  Loss: 1.5769009590148926
Epoch 8  Loss: 1.461165428161621
Epoch 9  Loss: 1.4612427949905396


 <h2>Testing the network</h2>
 
 We should just loop through the train set and get the network's outputs.

In [57]:
correct_preds = 0
all_preds     = 0

for curr_batch in testloader:
    vectors, labels = curr_batch[0].to(device), curr_batch[1].to(device)
    
    predictions = torch.argmax(net(vectors), dim=1)
    
    all_preds     += predictions.size(0)
    correct_preds += (predictions == labels).sum().item()

print("Accuracy is", (correct_preds / all_preds) * 100, "per cent")

Accuracy is 97.85000000000001 per cent


<h2>Conclusion</h2>

I developed a fully-functional MNIST Classifier. In this classifier we can see that even with a simple convolutional network we get a pretty good accuracy.